<a href="https://colab.research.google.com/github/Felipanjos/a3_ia_2022.2/blob/main/Projeto_IA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2.2 Requisitos da Análise do problema

### A. Entendimento do Problema:

#### a. Especificação do objetivo geral e dos objetivos específicos

Para esse projeto utilizamos os conhecimentos adquiridos ao longo da nossa disciplina, para criar uma Maquina Preditiva com o objetivo de estimar quantos transistores existem em um determinado CPU. Sabemos que mais transistores em nossa CPU significa mais paralelismo para as tarefas do mesmo, permitindo que essas tarefas sejam finalizadas mais rapidamente. Historicamente vimos que os processadores se tornaram cada vez mais poderosos e junto a isso podemos perceber que o numero de transistores aumentou significativamente. 

## B. Enquadramento do Problema:

### a. Especificação do tipo de problema: Quanto à Supervisão, Quanto à Tarefa, Quanto ao Modo;

Para esse projeto utilizamos os conhecimentos adquiridos ao longo da nossa disciplina, para criar uma Maquina Preditiva com o objetivo de estimar quantos transistores existem em um determinado CPU. Sabemos que mais transistores em nossa CPU significa mais paralelismo para as tarefas do mesmo, permitindo que essas tarefas sejam finalizadas mais rapidamente. Historicamente vimos que os processadores se tornaram cada vez mais poderosos e junto a isso podemos perceber que o numero de transistores aumentou significativamente.


b. Especificação das Medidas de Desempenho (para Classificação ou Regressão)


Como metódos de desempenho, os principais utilizados no nosso projeto, se tratam de acurácia para a Classicação e erro quadrático médio para Regressão, esses metódos, são extremamente importantes, pois é através deles que é possível observar o quão precisas estão sendo as predições realizadas pela máquina, por serem duas formas diferentes de Aprendizado de máquina supervisionado, cada um possui seu próprio método de desempenho, para a classificação usaremos o principal citado anteriormente, acurácia, se trata de um indicador da performance geral do modelo, contando quantas classificações foram feitas corretamente, para ser calculado, se torna necessário utilizar a Matriz de confusão, identificando os verdadeiros positivos, falso negativos, falso positivos e verdadeiros negativos, através desses dados é possível montar a Matriz e utilizar a formúla VP+VN/VP+VN+FP+FN para assim medir a acurácia do modelo, já para a regressão utilizaremos o erro quadrático médio, esse método carrega como objetivo encontrar a diferença dos valores iniciais com os valores preditos, quanto mais alta a diferença, maior o problema do modelo, pois os valores encontrados não estão sendo precisos, essa média é calculada através da diferença do valor predito para o valor real, o resultado é elevado ao quadrado, fazendo isso para todos os pontos, como já dito, quanto maior o resultado, mais pobre estão sendo as predições.

# 2.3 Requisitos da Análise de Dados:

In [ ]:
import pandas as pd
import numpy as np
import random
import string
import sklearn
from itertools import groupby

from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns
import statistics as sts

In [ ]:
df = pd.read_csv('chip_dataset.csv', index_col=[0])
df.drop('FP16 GFLOPS', axis=1, inplace=True)
df.drop('FP32 GFLOPS', axis=1, inplace=True)
df.drop('FP64 GFLOPS', axis=1, inplace=True)
vendors = list(np.unique(df['Vendor']))

In [ ]:
num_attribs = list(df.describe().columns)
cat_attribs = ['Vendor']

In [ ]:
df.head()

### B. Divisão

##### a) Amostragem estratificada em Conjunto de Treinamento e Conjunto de Teste

In [ ]:
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

In [ ]:
ranges = [0, df['Transistors (million)'].describe()['25%'], 500, 1000, 2000, 6000, 15000, 30000, np.inf]
tiers = ['F', 'E', 'D', 'C', 'B', 'A', 'S', 'S+']

df['Rank'] = pd.cut(df['Transistors (million)'], bins = ranges, labels = tiers)

In [ ]:
splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in splitter.split(df, df['Rank']):
  strat_train_set = df.loc[train_index]
  strat_test_set = df.loc[test_index]

In [ ]:
def amostragem_estratificada(test_set):
    return (test_set['Rank'].value_counts() / len(test_set))*100    

In [ ]:
amostragem_estratificada(strat_test_set)

In [ ]:
strat_train_set.drop(columns=['Rank'], inplace=True)
strat_test_set.drop(columns=['Rank'], inplace=True)
cpu = strat_train_set.drop('Transistors (million)', axis=1)

### C. Exploração 

##### a) Visualização dos Dados com gráficos de dispersão e histograma

In [ ]:
df['Release Date'] = pd.to_datetime(df['Release Date'])

In [ ]:
# ampliar escala 
strat_test_set['Transistors (million)'].hist(figsize=(10, 8))

In [ ]:
strat_train_set['Transistors (million)'].hist()

In [ ]:
df.plot(kind='scatter', x='Release Date', y='Transistors (million)')

##### b. Levantamento de hipóteses sobre as distribuições dos dados


##### c. Busca de correlações (Coeficientes de correlação)

Nesta abordagem inicial do projeto, buscamos conectar o número de transistores com o sua Potência térmica máxima (TDP)  e o tamanho de sua matriz (Die size) devido a  avaliação positiva dos coeficientes de relação e a sua lógica primordial, quanto mais transistores normalmente é apresentada uma TDP mais elevada e uma matriz maior.

In [ ]:
df['Release Date'] = pd.to_numeric(df['Release Date'])

##### Basearemos a análise das valueelações na coluna Transistors (million), que é o nosso objeto de predição

In [ ]:
for value in df.corr()['Transistors (million)']:
    print(value)

In [ ]:
colunas_para_calculo = ['Release Date'] + num_attribs
print('Coeficiente de correlação de Pearson entre a coluna Transistors (million) e:')

coor_dict = {}
r_list = []

for value in df.corr()['Transistors (million)']:
    if value == 1:
        r = 'relação linear perfeita e positiva'
    elif value == 0:
        r = 'inexistência linear'
    elif value == -1:
        r = 'relação linear perfeita e negativa'
    elif value > 0:
        r = 'relação linear positiva'
    elif value < 0:
        r = 'relação linear negativa'

    r_list.append(r)

print(df.corr(method='pearson', min_periods=1)['Transistors (million)'], '\n')

count = 0
for col in colunas_para_calculo: 
    coor_dict[col] = r_list[count]
    print(f'{col}: {coor_dict[col]}')
    count += 1


### D. Preparação

##### a) Selecionando 10% das colunas

In [ ]:
qtd_colunas = len(df.columns)
p_colunas = round(qtd_colunas * 10 / 100)
coluna_aleatoria = df.sample(n=p_colunas, axis='columns', random_state=1).keys()[0]
colunas_numericas = df.describe().columns
colunas_categoricas = df.describe(exclude=np.number).columns
coluna_aleatoria # escolhendo uma coluna aleatoriamente

##### a) Selecionando 3% dos dados

In [ ]:
len(df)

In [ ]:
p_3 = round(df.shape[0] * 3 / 100)
p_3

##### a) Excluindo 3% dos dados aleatoriamente

In [ ]:
reg_excluidos = df.sample(n=p_3)
reg_exc_index = reg_excluidos.index.array
df.loc[reg_exc_index, coluna_aleatoria] = np.nan

##### a) Alterando 3% dos dados aleatoriamente

In [ ]:
def gerar_string_aleatoria():
  letters = string.ascii_lowercase
  return ''.join(random.choice(letters) for i in range(10)) 

def gerar_numero_aleatorio():
  return random.randint(0, 999)

In [ ]:
# listando indexes que sofreram alteração na coluna aleatória
df_selecao_nao_nulo = df[~df[coluna_aleatoria].isnull()]
reg_alterados = df_selecao_nao_nulo.sample(n=p_3)
reg_alter_index = reg_alterados.index.array

In [ ]:
if coluna_aleatoria in colunas_numericas:
    for index in reg_alter_index:
        df.loc[[index],[coluna_aleatoria]] = gerar_numero_aleatorio()
else:
    for index in reg_alter_index:
        df.loc[[index],[coluna_aleatoria]] = gerar_string_aleatoria()

In [ ]:
# listando alterações
count = 0
for index in reg_alter_index:
    print(df[coluna_aleatoria][index])
    count += 1
    if count == 10:
        break

#### b) Limpeza dos dados

i. Codificação One-Hot

In [ ]:
encoder = OneHotEncoder()
onehot_encoded_df = pd.DataFrame(encoder.fit_transform(df[['Vendor']]).toarray())
onehot_encoded_df.columns = vendors

onehot_encoded_df = df.join(onehot_encoded_df)
onehot_encoded_df.drop('Vendor', axis=1, inplace=True)
onehot_encoded_df

ii. Estratégia para dados numéricos ausentes - mediana geral

In [ ]:
df.describe()

In [ ]:
sts.median(df['Transistors (million)'])

A atribuição da mediana geral foi utilizada para lidar com os valores nulos das colunas em que existiam, pelos seguintes motivos:


Utilizaremos a coluna 'Transistors (million)' como referência por acreditarmos que se trata da coluna principal do dataset.

*   A remoção dos valores nulos do conjunto levaria à retirada de 29.1% dos dados, pouco mais de 1/4 do dataset original
*   Pela forte presença de outliers no dataset, o uso da média para atribuição de dados ausentes não seria uma boa opção. A média da coluna 'Transistors (million)'     seria aproximadamente 2151 milhões.
*   O treinamento de um preditor geraria esforço adicional e sairia do foco da entrega, então utilizamos uma estratégia mais simples para tratar dessa questão.
*   Por isso, escolhemos o uso da mediana. Para a coluna 'Transistorts (millions)' a mediana é 731 milhões, praticamente 3 vezes menos do que a média. A partir disso, obtemos o valor central do dataset, que sofre menos com o impacto dos outliers.


iii. Escalonamento de características: Padronização e iv. Pipeline transformadora

In [ ]:
num_attribs = list(cpu.describe(include=np.number))

In [ ]:
num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),        
        ('std_scaler', StandardScaler()),
    ])

full_pipeline = ColumnTransformer([
        ('num', num_pipeline, num_attribs),
        ('cat', OneHotEncoder(), cat_attribs),
    ])

In [ ]:
cpu

In [ ]:
cpu_prepared = full_pipeline.fit_transform(cpu)

Utilizando a full_pipeline, aplicamos
*   Codificação One-Hot para dados categóricos com mais de duas categorias (escolhemos a coluna 'Vendor')
*   Atribuição da mediana geral para os dados numéricos ausentes
*   Escalonamento de caretísticas: Padronização
*   Pipeline transformadora utilizando codificaçõa one hot, atribuição de mediana e padronização


In [ ]:
cpu_prepared

##### Treinamento do modelo

In [240]:
cpu_labels = strat_train_set['Transistors (million)'].copy()

lin_reg = LinearRegression()
lin_reg.fit(cpu_prepared, cpu_labels)

LinearRegression()

In [241]:
some_data = cpu.iloc[:5]
some_labels = cpu_labels.iloc[:5]
some_data_prepared = full_pipeline.transform(some_data)

In [242]:
print("Predictions:", lin_reg.predict(some_data_prepared))

Predictions: [ 2239.17928815 -1256.94052508   845.05035726   289.9519808
  3657.93900242]


In [243]:
print("Labels:", list(some_labels))

Labels: [1040.0, 82.0, 486.0, 582.0, 1178.0]


In [244]:
from sklearn.metrics import mean_squared_error

cpu_predictions = lin_reg.predict(cpu_prepared)
lin_rmse = mean_squared_error(cpu_labels, cpu_predictions, squared=False)
lin_rmse

3074.6307079120093